## Set system path

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/DLC-Jupyter-Notebooks
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import os
import psycopg2
import pandas as pd
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(dbname="deeplabcut_db", user="postgres", password="1234", host=host, port="5432")
cursor = conn.cursor()


## Extract IDs

In [ ]:
FoodOnly_saline_q = "SELECT id FROM dlc_table WHERE task='FoodOnly' AND health='saline' AND trial_length>1200;"
df = pd.read_sql_query(FoodOnly_saline_q, conn)
FoodOnly_saline_id = df['id'].to_list()

FoodOnly_ghrelin_q = "SELECT id FROM dlc_table WHERE task='FoodOnly' AND health='ghrelin' AND trial_length>1200;"
df = pd.read_sql_query(FoodOnly_ghrelin_q, conn)
FoodOnly_ghrelin_id = df['id'].to_list()


LightOnly_saline_q = "SELECT id FROM dlc_table WHERE task='LightOnly' AND health='saline' AND trial_length>1200;"
df = pd.read_sql_query(LightOnly_saline_q, conn)
LightOnly_saline_id = df['id'].to_list()

LightOnly_ghrelin_q = "SELECT id FROM dlc_table WHERE task='LightOnly' AND health='ghrelin' AND trial_length>1200;"
df = pd.read_sql_query(LightOnly_ghrelin_q, conn)
LightOnly_ghrelin_id = df['id'].to_list()


ToyOnly_saline_q = "SELECT id FROM dlc_table WHERE task='ToyOnly' AND health='saline' AND trial_length>1200;"
df = pd.read_sql_query(ToyOnly_saline_q, conn)
ToyOnly_saline_id = df['id'].to_list()

ToyOnly_ghrelin_q = "SELECT id FROM dlc_table WHERE task='ToyOnly' AND health='ghrelin' AND trial_length>1200;"
df = pd.read_sql_query(ToyOnly_ghrelin_q, conn)
ToyOnly_ghrelin_id = df['id'].to_list()


ToyLight_saline_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='saline' AND trial_length>1200;"
df = pd.read_sql_query(ToyLight_saline_q, conn)
ToyLight_saline_id = df['id'].to_list()

ToyLight_ghrelin_q = "SELECT id FROM dlc_table WHERE task='ToyLight' AND health='ghrelin' AND trial_length>1200;"
df = pd.read_sql_query(ToyLight_ghrelin_q, conn)
ToyLight_ghrelin_id = df['id'].to_list()

## Total distance

In [ ]:
import importlib
import Python_scripts.Data_analysis.get_total_and_mean_feature_per_id
import Python_scripts.Data_analysis.plot_groupwise_bar

importlib.reload(Python_scripts.Data_analysis.get_total_and_mean_feature_per_id)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)

from Python_scripts.Data_analysis.get_total_and_mean_feature_per_id import get_total_and_mean_feature_per_id
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar


In [ ]:
df_sal = get_total_and_mean_feature_per_id(
    conn, FoodOnly_saline_id, 'saline', 'distance', max_time=1200)
df_ghr = get_total_and_mean_feature_per_id(
    conn, FoodOnly_ghrelin_id, 'ghrelin', 'distance', max_time=1200)
df_total = pd.concat([df_sal, df_ghr], ignore_index=True)

print(df_total.head(2))

plot_groupwise_bar(
    df_total, y='total_distance', title='ToyLight', ylabel='Total Distance')


## Binned distance

In [ ]:
import importlib
import Python_scripts.Data_analysis.compute_binned_array_feature_stats_all
import Python_scripts.Data_analysis.plot_binned_feature_with_sem_from_raw

importlib.reload(Python_scripts.Data_analysis.compute_binned_array_feature_stats_all)
importlib.reload(Python_scripts.Data_analysis.plot_binned_feature_with_sem_from_raw)

from Python_scripts.Data_analysis.compute_binned_array_feature_stats_all import compute_binned_array_feature_stats_all
from Python_scripts.Data_analysis.plot_binned_feature_with_sem_from_raw import plot_binned_feature_with_sem_from_raw


In [ ]:
df_sal_bins = compute_binned_array_feature_stats_all(
    conn, FoodOnly_saline_id, 'distance', 'saline',
    time_limit=1200.0, bin_size=120, t_offset=1)

df_ghr_bins = compute_binned_array_feature_stats_all(
    conn, FoodOnly_ghrelin_id, 'distance', 'ghrelin',
    time_limit=1200.0, bin_size=120, t_offset=1)

print(df_sal_bins.head(2))

df_all_bins = pd.concat([df_sal_bins, df_ghr_bins], ignore_index=True)

plot_binned_feature_with_sem_from_raw(
    df_all_bins,
    feature='distance_sum',
    title='ToyLight',
    ylabel='Distance (sum ± SEM)'
)
